# GrimAge2LogCRP

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [34]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import numpy as np

## Instantiate model class

In [35]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.GrimAge2LogCRP)

class GrimAge2LogCRP(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x
    
    def postprocess(self, x):
        return x



In [36]:
model = pya.models.GrimAge2LogCRP()

## Define clock metadata

In [37]:
model.metadata["clock_name"] = 'grimage2logcrp'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2022
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Lu, Ake T., et al. \"DNA methylation GrimAge version 2.\" Aging (Albany NY) 14.23 (2022): 9484."
model.metadata["doi"] = "https://doi.org/10.18632/aging.204434"
model.metadata["research_only"] = True
model.metadata["notes"] = None

## Download clock dependencies

In [38]:
logger = pya.logger.Logger()
urls = [
    "https://pyaging.s3.amazonaws.com/supporting_files/grimage2_subcomponents.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/grimage2.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/datMiniAnnotation3_Gold.csv",
]
dir = "."
for url in urls:
    pya.utils.download(url, dir, logger, indent_level=1)

|-----------> Downloading data to ./grimage2_subcomponents.csv
|-----------> in progress: 100.0000%
|-----------> Downloading data to ./grimage2.csv
|-----------> in progress: 100.0000%%
|-----------> Downloading data to ./datMiniAnnotation3_Gold.csv
|-----------> in progress: 100.0000%


## Load features

#### From CSV

In [39]:
df = pd.read_csv('grimage2_subcomponents.csv', index_col=0)
df['Y.pred'].unique()

array(['DNAmGDF_15', 'DNAmB2M', 'DNAmCystatin_C', 'DNAmTIMP_1', 'DNAmadm',
       'DNAmpai_1', 'DNAmleptin', 'DNAmPACKYRS', 'DNAmlog.CRP',
       'DNAmlog.A1C'], dtype=object)

In [40]:
df = df[df['Y.pred'] == 'DNAmlog.CRP']
df['feature'] = df['var']
df['coefficient'] = df['beta']
model.features = df['feature'][1:].tolist()

In [41]:
df.head()

,Y.pred,var,beta,feature,coefficient
1796,DNAmlog.CRP,Intercept,-0.231448,Intercept,-0.231448
1797,DNAmlog.CRP,cg01391867,-0.238114,cg01391867,-0.238114
1798,DNAmlog.CRP,cg06298190,0.137911,cg06298190,0.137911
1799,DNAmlog.CRP,cg03110633,0.397843,cg03110633,0.397843
1800,DNAmlog.CRP,cg05303999,0.445975,cg05303999,0.445975


## Load weights into base model

#### Linear model

In [42]:
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'].iloc[0]])

#### Linear model

In [43]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [44]:
reference_df = pd.read_csv('datMiniAnnotation3_Gold.csv', index_col=0)
model.reference_values = reference_df.loc[model.features[0:]]['gold'].tolist()

## Load preprocess and postprocess objects

In [45]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [46]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [47]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Lu, Ake T., et al. "DNA methylation GrimAge version 2." Aging '
             '(Albany NY) 14.23 (2022): 9484.',
 'clock_name': 'grimage2logcrp',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.18632/aging.204434',
 'notes': None,
 'research_only': True,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2022}
reference_values: [0.0229199108411034, 0.18157609704669, 0.0145712286188007, 0.913922509481157, 0.809605421222521, 0.703858551128724, 0.828079343471397, 0.8354705752375, 0.945562513470214, 0.219544227570287, 0.628295804305801, 0.0556920557050531, 0.883510168741842, 0.80868027489568, 0.942843729694158, 0.175324342316158, 0.873109700042845, 0.0947894779919567, 0.733082005089516, 0.883065598559295, 0.127222466210419, 0.846006900277736, 0.323099124395025, 0.0672194476736834, 0.919339370837926, 0.09

## Basic test

In [48]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float).double()
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ -1.5050],
        [ -5.0319],
        [  1.0132],
        [-24.1760],
        [ -8.6024],
        [ -5.2143],
        [ 13.2554],
        [ -3.3618],
        [-10.8422],
        [ -6.0286]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [49]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [50]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: grimage2_subcomponents.csv
Deleted file: datMiniAnnotation3_Gold.csv
Deleted file: grimage2.csv
